In [17]:
# Import required libraries
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Read CSV directly from URL, no need to download locally
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Initialize Dash application
app = dash.Dash(__name__)

# Application layout (same as before)
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Data Analysis Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),

    # Task 1: Add launch site dropdown component
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Launch Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
        ],
        value='ALL',
        placeholder="Please choose the launch site",
        searchable=True
    ),
    html.Br(),

    # Task 2: Success launch pie chart
    dcc.Graph(id='success-pie-chart'),
    html.Br(),

    html.P("Payload Mass Range (kg):"),
    # Task 3: Add payload mass range slider
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={
            0: '0',
            2000: '2000',
            4000: '4000',
            6000: '6000',
            8000: '8000',
            10000: '10000'
        },
        value=[min_payload, max_payload]
    ),
    html.Br(),

    # Task 4: Payload vs Success Rate scatter chart
    dcc.Graph(id='success-payload-scatter-chart'),
])

# Task 2: Callback function - Render pie chart (same as before)
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    # Filter data
    filtered_df = spacex_df
    if entered_site == 'ALL':
        # All sites: Count total successful launches by each launch site
        fig = px.pie(
            filtered_df,
            values='class',
            names='Launch Site',
            title='Success Launch Proportion of All Launch Sites'
        )
    else:
        # Specific site: Count success (1) and failure (0) launches for the selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(
            filtered_df,
            values=filtered_df['class'].value_counts().values,
            names=filtered_df['class'].value_counts().index,
            title=f'Success/Failure Proportion of {entered_site} (1=Success, 0=Failure)'
        )
    return fig

# Task 4: Callback function - Render scatter chart (same as before)
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def get_scatter_chart(entered_site, payload_range):
    # Unpack payload range
    low, high = payload_range
    # Filter data by payload mass first
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & 
                            (spacex_df['Payload Mass (kg)'] <= high)]
    
    # Filter data by launch site
    if entered_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
    
    # Draw scatter chart
    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        title=f'Relationship between Payload Mass and Launch Success Rate of {entered_site if entered_site != "ALL" else "All Launch Sites"}',
        labels={'class': 'Launch Result (1=Success, 0=Failure)'}
    )
    return fig

# Run the application
if __name__ == '__main__':
    app.run(debug=True, port=8051)

1.KSC LC-39A
2.KSC LC-39A
3.2000-4000 kg  
4.6000-8000 kg
5. FT
The launch site with the highest number of successful launches is KSC LC-39A, which also has the highest launch success rate among all sites. The payload mass range with the highest launch success rate is 2000-4000 kg, while the range with the lowest success rate is 6000-8000 kg. The FT version of the Falcon 9 booster has the highest launch success rate.

In [11]:
import pandas as pd

# Read data from URL (no need to download locally)
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(url)

# 2. Define payload mass ranges (consistent with dashboard slider)
payload_ranges = [
    (0, 2000, "0-2000 kg"),
    (2000, 4000, "2000-4000 kg"),
    (4000, 6000, "4000-6000 kg"),
    (6000, 8000, "6000-8000 kg"),
    (8000, 10000, "8000-10000 kg")
]

# 3. Initialize result list to store statistics for each range
results = []

# 4. Calculate success rate for each payload range one by one
for min_payload, max_payload, range_name in payload_ranges:
    # Filter data for the current range
    filtered_df = spacex_df[
        (spacex_df['Payload Mass (kg)'] >= min_payload) & 
        (spacex_df['Payload Mass (kg)'] <= max_payload)
    ]
    
    # Count successful (class=1) and failed (class=0) launches
    total_launches = len(filtered_df)
    success_launches = len(filtered_df[filtered_df['class'] == 1])
    fail_launches = len(filtered_df[filtered_df['class'] == 0])
    
    # Calculate success rate (avoid division by zero, set to 0 if no data)
    success_rate = (success_launches / total_launches * 100) if total_launches > 0 else 0
    
    # Save results
    results.append({
        "Payload Range": range_name,
        "Total Launches": total_launches,
        "Successful Launches": success_launches,
        "Failed Launches": fail_launches,
        "Success Rate (%)": round(success_rate, 2)  # Keep 2 decimal places
    })

# 5. Convert to DataFrame for easy viewing and sorting
results_df = pd.DataFrame(results)

# 6. Print results
print("===== Launch Success Rate by Payload Mass Range =====")
print(results_df)

# 7. Find the range with the highest and lowest success rate
max_rate_row = results_df.loc[results_df['Success Rate (%)'].idxmax()]
min_rate_row = results_df.loc[results_df['Success Rate (%)'].idxmin()]

print("\n===== Key Conclusions =====")
print(f"✅ Highest Success Rate Range: {max_rate_row['Payload Range']}, Success Rate {max_rate_row['Success Rate (%)']}%")
print(f"❌ Lowest Success Rate Range: {min_rate_row['Payload Range']}, Success Rate {min_rate_row['Success Rate (%)']}%")

===== Launch Success Rate by Payload Mass Range =====
   Payload Range  Total Launches  Successful Launches  Failed Launches  \
0      0-2000 kg              13                    3               10   
1   2000-4000 kg              21                   13                8   
2   4000-6000 kg              13                    5                8   
3   6000-8000 kg               4                    0                4   
4  8000-10000 kg               5                    3                2   

   Success Rate (%)  
0             23.08  
1             61.90  
2             38.46  
3              0.00  
4             60.00  

===== Key Conclusions =====
✅ Highest Success Rate Range: 2000-4000 kg, Success Rate 61.9%
❌ Lowest Success Rate Range: 6000-8000 kg, Success Rate 0.0%
